<a href="https://colab.research.google.com/github/DianeMack22/econ8310-assignment3/blob/main/Test_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -----------------------------
# 1. Import dependencies and custom code
# -----------------------------
import torch
from torch.utils.data import DataLoader
from assignment3.py import CustomMNIST, FirstNet

# -----------------------------
# 2. Load the test dataset
# -----------------------------
test_dataset = CustomMNIST(
    image_path="data/t10k-images-idx3-ubyte",
    label_path="data/t10k-labels-idx1-ubyte"
)

test_loader = DataLoader(test_dataset, batch_size=64)

# -----------------------------
# 3. Load the trained model
# -----------------------------
model = FirstNet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Load checkpoint
checkpoint = torch.load("model1.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

# -----------------------------
# 4. Evaluate the model on test data
# -----------------------------
correct = 0
total = 0

with torch.no_grad():
    for X, y in test_loader:
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

accuracy = 100 * correct / total
print(f"✅ Model accuracy on test set: {accuracy:.2f}%")

ModuleNotFoundError: No module named 'assignment3'